In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! ls drive/My\ Drive/Flow\ Identification\ Data

'Annular 25 mm'  'Full Flow'  'Qtr Flow'
'Annular 75 mm'  'Half Flow'  'Three Qtr Flow'


In [0]:
BASE_BATH = r"drive/My Drive/Flow Identification Data"

In [0]:
mapper = {'Annular 25 mm':0 , 'Full Flow':1 , 'Qtr Flow':2 , 'Annular 75 mm':3 , 'Half Flow':4 , 'Three Qtr Flow':5}

In [0]:
import os 
import cv2
labels = []
image_container = []
for folder_name in os.listdir(BASE_BATH):
  folder_path = os.path.join(BASE_BATH , folder_name)
  counter = 0
  for image_name in os.listdir(folder_path):
    counter += 1
    if counter ==3:
      image_path = os.path.join(folder_path , image_name)
      image = cv2.imread(image_path)
      image = cv2.resize(image, (96, 96))
      image = image.astype("float") / 255.0
      image_container.append(image)
      labels.append(mapper[folder_name])
      counter = 0


In [0]:
import numpy as np
from keras.utils import np_utils
np.random.seed(42)
image_container = np.array(image_container,dtype=np.float32)
image_container.reshape(len(image_container), 96, 96 , 3)
train_labels = np.array(labels,dtype=np.uint8)


Using TensorFlow backend.


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_container, train_labels, test_size=0.1, shuffle=True)

In [0]:
Y_train = np_utils.to_categorical(y_train, 6)
Y_test = np_utils.to_categorical(y_test, 6)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

#creat cnn model
model = Sequential()
#add first conv layer
model.add(Convolution2D(32, (3, 3), activation='relu',  padding="same",
                 input_shape=(96,96,3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Convolution2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(6, activation='softmax'))


In [0]:
from keras.metrics import categorical_accuracy
from keras.optimizers import Adam, Adadelta
model.compile(Adadelta(lr=0.2, decay=0.0001),loss="categorical_crossentropy", metrics=["accuracy"])

In [0]:

model.fit(X_train, Y_train,validation_data=(X_test,Y_test) , batch_size=32, nb_epoch=10, verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 9736 samples, validate on 1082 samples
Epoch 1/10
9736/9736 [==============================] - 350s 36ms/step - loss: 0.0620 - accuracy: 0.9798 - val_loss: 4.8389e-04 - val_accuracy: 1.0000
Epoch 2/10
9736/9736 [==============================] - 348s 36ms/step - loss: 1.9506e-04 - accuracy: 1.0000 - val_loss: 8.3411e-05 - val_accuracy: 1.0000
Epoch 3/10
9736/9736 [==============================] - 346s 36ms/step - loss: 5.5276e-05 - accuracy: 1.0000 - val_loss: 4.2979e-05 - val_accuracy: 1.0000
Epoch 4/10
9736/9736 [==============================] - 347s 36ms/step - loss: 3.1075e-05 - accuracy: 1.0000 - val_loss: 2.7150e-05 - val_accuracy: 1.0000
Epoch 5/10
9736/9736 [==============================] - 346s 36ms/step - loss: 2.1425e-05 - accuracy: 1.0000 - val_loss: 1.9759e-05 - val_accuracy: 1.0000
Epoch 6/10
9736/9736 [==============================] - 347s 36ms/step - loss: 1.6351e-05 - accuracy: 1.0000 - val_loss: 1.5287e-05 - val_accuracy: 1.0000
Epoch 7/10
9736/9736 [====

In [0]:
model.save("flow_identification_model.h5")

In [0]:
!cp -r flow_identification_model.h5 drive/My\ Drive/flow_identification_model.h5 
